## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
# These columns are not relevant to the machine learning model because they are unique identifiers
# and do not contribute to predicting the target variable (IS_SUCCESSFUL).
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

# Display the first few rows to confirm the columns were dropped
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
# This helps us understand the distribution and variety of values in each feature,
# which is important for preprocessing and encoding decisions.
unique_values = application_df.nunique()

# Display the number of unique values for each column to identify potential high-cardinality features
unique_values

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
# We need to identify rare application types that might add noise to the model.
# This step will help us reduce the number of categories by grouping less common types together.
app_type_counts = application_df["APPLICATION_TYPE"].value_counts()

# Display the counts to understand the distribution of application types
app_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# We will group less common application types as "Other" to reduce model complexity and noise.
# The cutoff is typically chosen to keep the most common types and combine the rest.
application_types_to_replace = list(app_type_counts[app_type_counts < 500].index)

# Replace in dataframe
# Loop through each application type in the replacement list and update the dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
# Like with APPLICATION_TYPE, we need to reduce the number of unique CLASSIFICATION categories.
# This helps avoid high cardinality and overfitting in the model.
classification_counts = application_df["CLASSIFICATION"].value_counts()

# Display the counts to identify rare classifications for potential grouping
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# This filters the classification counts to only include those that have more than one occurrence.
# This helps us identify the more common classifications and focus on rare types for grouping.
classification_counts[classification_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# Use the variable name `classifications_to_replace`
# We are grouping less common classifications into the "Other" category to reduce noise and model complexity.
# The cutoff is typically chosen based on the distribution to capture the majority while reducing sparsity.
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
# Loop through each classification in the replacement list and update the dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
# This step transforms all categorical columns into numeric format, which is required for machine learning models.
# Each unique value in a categorical column becomes a separate binary (0/1) column.
application_df = pd.get_dummies(application_df)

# Display the first few rows to confirm the conversion
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [11]:
# Split our preprocessed data into our features and target arrays
# The target array (y) is the 'IS_SUCCESSFUL' column, which we want to predict.
# The features array (X) includes all remaining columns.
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
# We use an 80-20 split to train the model on the majority of the data while reserving a portion for testing.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Display the shapes of the resulting datasets to confirm the split
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((27439, 43), (6860, 43), (27439,), (6860,))

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# The input layer must match the number of features in the training data.
number_input_features = len(X_train_scaled[0])

# Defining the model as a Sequential neural network
nn = tf.keras.models.Sequential()

# First hidden layer
# This layer has 80 nodes (a reasonable starting point for a deep neural net) and uses the ReLU activation function,
# which is commonly used for hidden layers to introduce non-linearity.
nn.add(tf.keras.layers.Dense(units=80, input_dim=number_input_features, activation="relu"))

# Second hidden layer
# Adding a second hidden layer with 30 nodes and ReLU activation to increase the model's learning capacity.
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
# The output layer has a single node with a sigmoid activation function,
# which is appropriate for binary classification problems.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/opt/anaconda3/envs/dev/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Compile the model
# Compiling the model with the optimizer, loss function, and evaluation metric.
# - The optimizer 'adam' is widely used for deep learning as it adjusts the learning rate dynamically.
# - The loss function 'binary_crossentropy' is appropriate for binary classification.
# - The evaluation metric 'accuracy' will track the model's performance during training.
nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [15]:
# Train the model
# Training the model on the scaled training data with a specified number of epochs.
# - Using 100 epochs to match the sample output.
# - Setting a batch size of 32 for efficient gradient updates.
history = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32)

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 298us/step - accuracy: 0.7011 - loss: 0.5891
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 289us/step - accuracy: 0.7276 - loss: 0.5548
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 291us/step - accuracy: 0.7287 - loss: 0.5547
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 289us/step - accuracy: 0.7317 - loss: 0.5474
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 288us/step - accuracy: 0.7294 - loss: 0.5503
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 289us/step - accuracy: 0.7334 - loss: 0.5473
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 288us/step - accuracy: 0.7380 - loss: 0.5416
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 316us/step - accuracy: 0.7375 - loss: 0.5423
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 289us/step - accuracy: 0.7341 - loss: 0.5428
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 286us/step - accuracy: 0.7351 - loss: 0.5456
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 287us/step - accuracy: 0.7387 - loss: 0.5408
Epoch 12/100
858/85

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 482us/step - accuracy: 0.7290 - loss: 0.5625
Loss: 0.5625095963478088, Accuracy: 0.7290087342262268


In [17]:
# Export our model to HDF5 file
# This saves the trained model to a file, allowing us to load it later without retraining.
# The file is named "AlphabetSoupCharity.h5" as specified in the instructions.
nn.save("AlphabetSoupCharity.h5")